# Data Cleaning

## Cleaning Steps
Document every cleaning step here.

In [1]:
import pandas as pd, pathlib, ast
from pandas import json_normalize

RAW = pathlib.Path('../data/raw/stops.json')
OUT_XLSX = pathlib.Path('../data/clean/stops_clean.xlsx')
OUT_CSV  = pathlib.Path('../data/clean/stops_clean.csv')

# 1) Načtení a rozbalení
df_raw = pd.read_json(RAW)
df = json_normalize(df_raw['stopGroups'])

# 2) Sloupce – sjednocení názvů
# stop_name
if 'stop_name' not in df.columns and 'name' in df.columns:
    df = df.rename(columns={'name': 'stop_name'})
# district_code
if 'district_code' not in df.columns and 'districtCode' in df.columns:
    df = df.rename(columns={'districtCode': 'district_code'})

# 3) Souřadnice → lat/lon
if {'avgLat','avgLon'}.issubset(df.columns):
    df = df.rename(columns={'avgLat': 'lat', 'avgLon': 'lon'})
elif {'gps.latitude','gps.longitude'}.issubset(df.columns):
    df['lat'] = pd.to_numeric(df['gps.latitude'], errors='coerce')
    df['lon'] = pd.to_numeric(df['gps.longitude'], errors='coerce')

df['lat'] = pd.to_numeric(df.get('lat'), errors='coerce')
df['lon'] = pd.to_numeric(df.get('lon'), errors='coerce')

# 4) Deduplikace – chytře vyber ID sloupec
id_candidates = ['uniqueName', 'node', 'uuid', 'idosId', 'idosCode', 'cis', 'stop_name']
id_col = next((c for c in id_candidates if c in df.columns), None)
print("Dedup budu dělat podle:", id_col)
if id_col:
    before = len(df)
    df = df.drop_duplicates(subset=id_col)
    print(f"Odstraněno duplicit: {before - len(df)}")

# 5) Dopočty & typy
# Počet podzastávek ve skupině (pokud jsou k dispozici)
if 'stops' in df.columns:
    def _count_stops(x):
        if isinstance(x, list): return len(x)
        if isinstance(x, str):
            try:
                val = ast.literal_eval(x)
                return len(val) if isinstance(val, list) else pd.NA
            except Exception:
                return pd.NA
        return pd.NA
    df['numStops'] = df['stops'].apply(_count_stops)

# isTrain – když je téměř prázdné, dopočteme z mainTrafficType
if 'isTrain' not in df.columns or df['isTrain'].isna().mean() > 0.9:
    if 'mainTrafficType' in df.columns:
        df['isTrain'] = df['mainTrafficType'].astype(str).str.lower().eq('train')

# Překlop JTSK, pokud existují
for c in ['avgJtskX','avgJtskY','cis','node','idosCategory']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# 6) Drop řádků bez souřadnic (pro mapu a další použití)
rows_before = len(df)
df = df.dropna(subset=['lat','lon'])
print(f"Řádků bez souřadnic odstraněno: {rows_before - len(df)}")

# 7) Pořadí a výběr praktických sloupců pro klienta
prefer_id = id_col if id_col else None
cols_keep = [c for c in [
    prefer_id,
    'stop_name','fullName','uniqueName',
    'municipality','district_code','mainTrafficType','idosCategory',
    'lat','lon','avgJtskX','avgJtskY',
    'numStops'
] if c in df.columns]

df_clean = df[cols_keep].copy()
if prefer_id and prefer_id != 'stop_id':
    df_clean = df_clean.rename(columns={prefer_id: 'stop_id'})

# 8) Uložení
OUT_XLSX.parent.mkdir(parents=True, exist_ok=True)
df_clean.to_excel(OUT_XLSX, index=False)
df_clean.to_csv(OUT_CSV, index=False, encoding='utf-8')
print(f"✅ Saved:\n  • {OUT_XLSX.resolve()}\n  • {OUT_CSV.resolve()}")

# 9) Rychlý souhrn
print("shape:", df_clean.shape)
print("columns:", df_clean.columns.tolist()[:12], "…")
print(df_clean.head(5))


Dedup budu dělat podle: uniqueName
Odstraněno duplicit: 71
Řádků bez souřadnic odstraněno: 0
✅ Saved:
  • /workspaces/jupiter-dev/data/clean/stops_clean.xlsx
  • /workspaces/jupiter-dev/data/clean/stops_clean.csv
shape: (8161, 13)
columns: ['stop_id', 'stop_name', 'fullName', 'stop_id', 'municipality', 'district_code', 'mainTrafficType', 'idosCategory', 'lat', 'lon', 'avgJtskX', 'avgJtskY'] …
      stop_id   stop_name    fullName     stop_id municipality district_code  \
0      Adamov      Adamov      Adamov      Adamov       Adamov            KH   
1    Albertov    Albertov    Albertov    Albertov        Praha            AB   
2  Ametystová  Ametystová  Ametystová  Ametystová        Praha            AB   
3    Amforová    Amforová    Amforová    Amforová        Praha            AB   
4       Anděl       Anděl       Anděl       Anděl        Praha            AB   

  mainTrafficType  idosCategory        lat        lon    avgJtskX    avgJtskY  \
0             bus        301003  49.858105